In [1]:
import os
import pandas as pd
import joblib
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()


# Retrieve the keys from environment variables
uri = os.getenv("MONGODB_URI")
openai_api_key = os.getenv("OPENAI_API_KEY")

print(f"URI , Api_KEY{uri,openai_api_key}")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key
# Initialize the OpenAI model with GPT-4-turbo
llm = ChatOpenAI(temperature=0.5, model="gpt-4-turbo")

# Load your dataset
csv_file_path = "dataset/data.csv"
df = pd.read_csv(csv_file_path)

# Create the agent using the loaded dataset and LLM
agent_executor = create_csv_agent(llm, csv_file_path, allow_dangerous_code=True, verbose=True)

# Save the agent model using joblib
with open('trained_model.pkl', 'wb') as f:
    joblib.dump(agent_executor, f)

print("Agent model saved successfully!")



PicklingError: Can't pickle <function create_react_agent.<locals>.<lambda> at 0x00000196FF870D60>: it's not found as langchain.agents.react.agent.create_react_agent.<locals>.<lambda>

In [ ]:
# ---------------------------------------------------- Run the Trained Model -------------------------------------------------------------
import os
import tkinter as tk
from tkinter import scrolledtext
import pickle

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()


# Retrieve the keys from environment variables
uri = os.getenv("MONGODB_URI")
openai_api_key = os.getenv("OPENAI_API_KEY")

print(f"URI , Api_KEY{uri,openai_api_key}")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = openai_api_key

# Load the saved agent from the file
with open('trained_model.pkl', 'rb') as f:
    agent_executor = pickle.load(f)

# Function to handle the prompt and display the result
def handle_prompt():
    prompt = prompt_entry.get()  # Get the user's prompt from the input field
    result = agent_executor.invoke(prompt)  # Invoke the agent with the user's prompt
    
    # Assuming the result is in a dictionary, extract the plain text
    if isinstance(result, dict):
        output = result.get("output", "No output found")  # Modify 'output' if necessary
    else:
        output = result
    
    # Display the output in the output text box
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, output)

# Create the main application window
root = tk.Tk()
root.title("LangChain CSV Agent")

# Create and pack the input prompt label and entry
prompt_label = tk.Label(root, text="Enter your prompt:")
prompt_label.pack(pady=5)

prompt_entry = tk.Entry(root, width=80)
prompt_entry.pack(pady=5)

# Create and pack the submit button
submit_button = tk.Button(root, text="Submit", command=handle_prompt)
submit_button.pack(pady=5)

# Create and pack the output text box (with a scrollbar)
output_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=100, height=20)
output_text.pack(pady=10)

# Run the GUI application
root.mainloop()


In [2]:
# --------------------------------------- Testing Paramters to verify model answer accuracy -----------------------------------

import pandas as pd

# Load the dataset
df = pd.read_csv("dataset/data.csv")

number_of_entries = df[df['Fiscal Year'] == '2014-2015'].shape[0]

# Display the result
print(f"Number of entries for the fiscal year 2014-2015: {number_of_entries}")

# Get the most frequently mentioned item
most_frequent_item = df['Item Name'].value_counts().idxmax()

# Get the count of that item
most_frequent_count = df['Item Name'].value_counts().max()

# Display the result
print(f"The most frequently mentioned item is: {most_frequent_item} with {most_frequent_count} mentions")


# Count the occurrences of each item
item_counts = df['Item Name'].value_counts()

# Get the least 3 ordered items
least_three_items = item_counts.nsmallest(3)

# Display the result
print("The three least frequently ordered items are:")
print(least_three_items)


# Group by 'Item Name' and sum the 'Quantity' to get total sales per item
item_sales = df.groupby('Item Name')['Quantity'].sum().reset_index()

# Find the most sold item
most_sold = item_sales.loc[item_sales['Quantity'].idxmax()]
print("Most Sold Item:")
print(most_sold)

# Find the least sold item
least_sold = item_sales.loc[item_sales['Quantity'].idxmin()]
print("Least Sold Item:")
print(least_sold)

# Clean the 'Total Price' column by removing commas or non-numeric symbols and converting to numeric
df['Total Price'] = pd.to_numeric(df['Total Price'].replace('[\$,]', '', regex=True), errors='coerce')

# Convert the 'Creation Date' to datetime format
df['Creation Date'] = pd.to_datetime(df['Creation Date'], errors='coerce')

# Extract the quarter from the 'Creation Date' and combine it with the 'Fiscal Year'
df['Quarter'] = df['Creation Date'].dt.quarter

# Group by Fiscal Year and Quarter, then sum the total spending
quarterly_spending = df.groupby(['Fiscal Year', 'Quarter'])['Total Price'].sum().reset_index()

# Identify the quarter with the highest spending
max_spending = quarterly_spending.loc[quarterly_spending['Total Price'].idxmax()]

# Output the result
print(f"The quarter with the highest spending is Fiscal Year {max_spending['Fiscal Year']}, Quarter {max_spending['Quarter']}, with a total spending of {max_spending['Total Price']:.2f}.")

<>:48: SyntaxWarning: invalid escape sequence '\$'
<>:48: SyntaxWarning: invalid escape sequence '\$'
C:\Users\ABC\AppData\Local\Temp\ipykernel_15824\3843907702.py:48: SyntaxWarning: invalid escape sequence '\$'
  df['Total Price'] = pd.to_numeric(df['Total Price'].replace('[\$,]', '', regex=True), errors='coerce')


Number of entries for the fiscal year 2014-2015: 116537
The most frequently mentioned item is: Medical Supplies with 2916 mentions
The three least frequently ordered items are:
Item Name
Extron pc video interface              1
Water Damage Remediation and Repair    1
FSR for Cal-Access - Amdt #3           1
Name: count, dtype: int64
Most Sold Item:
Item Name    3-2019 #10 Double Window Envelope
Quantity                            70000000.0
Name: 7721, dtype: object
Least Sold Item:
Item Name    Beans (dried)
Quantity            0.0001
Name: 25939, dtype: object
The quarter with the highest spending is Fiscal Year 2014-2015, Quarter 2, with a total spending of 28804820727.65.
